This notebooks corrects the model predictions of the minimal and maximal model, due to a mistake I made in experiment 3 (multi-organ models).

I did not change the labels of the BCV data-set, so the output predictions labelled the pancreas 3 instead of 1. This is no problem in itself, except that I have a lot of automatic scripts which went wrong because of it.

So here, I change the predictions such that label 3 becomes label 1 and vice versa.

In [5]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nibabel as nib 
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
from util import management as mana
from util import constants as con

# Set task specifics
task = con.TASK_523
pred_task = con.TASK_501
config = con.CONFIG_FULL
trainer = con.CLASSIC
label_new = 1
labels_old = 3
ts_tr = "Ts"

# Set paths 
results_dir = "C:\\Users\\ikke_\\OneDrive\\Documenten\\Thesis\\Results"
res_path = os.path.join(results_dir, str(task), config, trainer, "inference", str(pred_task), f"images{ts_tr}")
output_path = os.path.join(res_path, "relabeled")

# Create relabeled directory
if not(os.path.exists(output_path)):
    os.mkdir(output_path)

c:\Users\ikke_\OneDrive\Documenten\Thesis\MasterThesis\Code


In [6]:
# Relabel each segmentation
for f in [i for i in os.listdir(res_path) if i.endswith("nii.gz")]:

    # Load label
    path = os.path.join(res_path, f)
    label_niff = nib.load(path)
    label = np.array(label_niff.dataobj)

    # Make the 3 a 1
    np.place(label, label == label_new, 999) # move current '1' to dummy
    np.place(label, label == label_old, label_new) # move current '3' to '1'
    np.place(label, label == 999, label_old) # move dummy to '3'

    # Save
    new_label = nib.Nifti1Image(label, label_niff.affine, label_niff.header)
    nib.save(new_label, os.path.join(output_path, f))